# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'data': {
        'd': 3, #degree
        'n': 15, #number of variables
        'monomial_vars': None, #int or None
        'laurent': False, #use Laurent polynomials (negative degree with up to -d)
        'neg_d': 0, #int or None
        'neg_d_prob': 0, #probability before other adjustments
        'sparsity': None,
        'sample_sparsity': 15,
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',
        'a_max': 1,
        'a_min': -1,
        'polynomial_data_size': 101, #number of polynomials to generate
        'noise': 0, #fraction of mean(abs(y_data)) 
        'noise_distrib': 'normal', #'normal', 'uniform'
        
        'shift_polynomial': False,
        
        'border_min': 0.2, # defines an intervall. Value is randomly chosen and defines the minimum gap between x_min / x_max and the outermost stationary points => two values (left and right gap will be generated per variable)
        'border_max': 0.4,
        'lower_degree_prob': 0.5, # probability that the degree of the whole polynomial will be reduced
        'a_random_prob': 0.5, # probability that a random generated function is used without adjustement
                
        'global_stationary_prob': 1, # probability that all variables are used for adjustement (0 recommended for higher number of variables)
        'bulge_min': 1, # bulge_min and bulge_max define an intervall of how much the function is bulged
        'bulge_max': 4,
        'min_variables_used': 2, # defines an Intervall of how many variables are used to get stationary points and therefore adjust the function
        'max_variables_used': 6,
        'max_monomials': 7, # maximum number of monomials, before adjusting the function (monomial of degree 0 is always defined, but is included in this number)
        'max_monomials_random': 10, #maximum number of monomials for random generated functions
        
        'same_training_all_lambda_nets': False,
    },
    'lambda_net': {
        'lambda_dataset_size': 5000, #number of samples per polynomial
    },    
    'computation':{
        'n_jobs': 10,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 1,   
    }
}


In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['computation'])

## Imports

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
tf.get_logger().setLevel('WARNING')
tf.autograph.set_verbosity(2)

from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter

In [4]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n] 

os.environ['CUDA_VISIBLE_DEVICES'] = gpu_numbers if use_gpu else ''

sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

#sys.path.append('..')

In [5]:
from utilities.utility_functions import flatten, rec_gen, gen_monomial_identifier_list

list_of_monomial_identifiers_extended = []

if laurent:
    variable_sets = [list(flatten([[_d for _d in range(d+1)], [-_d for _d in range(1, neg_d+1)]])) for _ in range(n)]
    list_of_monomial_identifiers_extended = rec_gen(variable_sets)    
        
    print('List length: ' + str(len(list_of_monomial_identifiers_extended)))
    #print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(sparsity))
    #print('Sparsity:' + str(sparsity))
    if len(list_of_monomial_identifiers_extended) < 500:
        print(list_of_monomial_identifiers_extended)     
        
    list_of_monomial_identifiers = []
    for monomial_identifier in tqdm(list_of_monomial_identifiers_extended):
        if np.sum(monomial_identifier) <= d:
            if monomial_vars == None or len(list(filter(lambda x: x != 0, monomial_identifier))) <= monomial_vars:
                list_of_monomial_identifiers.append(monomial_identifier)        
else:
    variable_list = ['x'+ str(i) for i in range(n)]
    list_of_monomial_identifiers = gen_monomial_identifier_list(variable_list, d, n)
            
print('List length: ' + str(len(list_of_monomial_identifiers)))
#print('Number of monomials in a polynomial with ' + str(n) + ' variables and degree ' + str(d) + ': ' + str(sparsity))
#print('Sparsity: ' + str(sparsity))
print(list_of_monomial_identifiers)


List length: 816
[[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################
config['data']['sparsity'] = nCr(config['data']['n']+config['data']['d'], config['data']['d']) if not laurent else len(list_of_monomial_identifiers)
#nCr(config['data']['n']+config['data']['d'], config['data']['d']) if not laurent else (config['data']['d']*2+1)**config['data']['n'] - np.sum([i for i in range(1, config['data']['d']+1)])#- math.factorial(config['data']['d'])
#nCr(config['data']['n']+config['data']['d'], config['data']['d']) if not laurent else nCr(config['data']['n']+config['data']['d']*2, config['data']['d']*2)

config['data']['sample_sparsity'] = config['data']['sparsity'] if config['data']['sample_sparsity'] == None else config['data']['sample_sparsity']
    
#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['data'])
globals().update(config['lambda_net'])
globals().update(config['computation'])

initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

<class 'KeyError'>
<class 'KeyError'>


In [7]:
print(path_identifier_polynomial_data)


poly_101_train_5000_var_15_d_3_negd_0_prob_0_spars_15_amin_-1_amax_1_xdist_uniform_noise_normal_0_diffX


In [8]:
##############DO NOT CHANGE###################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]
    
print('Variables: ' + str(n) + ' (' + variables + ')')
print('Degree: ' + str(d))
print('Sparsity: ' + str(sparsity)) 
print('Lambda-Net Dataset Size: ' + str(lambda_dataset_size))
print('I-Net Dataset Size: ' + str(polynomial_data_size))
      
print('Coefficient Range: ' + '[' + str(a_min) + ', ' + str(a_max) + ']')
print('Variable Range: ' + '[' + str(x_min) + ', ' + str(x_max) + ']')

Variables: 15 (abcdefghijklmno)
Degree: 3
Sparsity: 816
Lambda-Net Dataset Size: 5000
I-Net Dataset Size: 101
Coefficient Range: [-1, 1]
Variable Range: [0, 1]


# Function Generation

In [9]:
if shift_polynomial:
    all_sparsities = [nCr(n+i, i) for i in range(d+1)]
    start = perf_counter()
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
    list_of_polynomials = parallel(delayed(get_polynomial_basic)(all_sparsities)for i in range(polynomial_data_size))
    list_of_polynomials = np.array(list_of_polynomials).astype('float64')
    end = perf_counter()
    print('Zeit zum generieren:', end-start, 'Sekunden')
    print('Zeit zum generieren pro Funktion:', (end-start)/polynomial_data_size)
else:
    list_of_polynomials = np.random.uniform(low=a_min, high=a_max, size=(polynomial_data_size, sparsity))   
        
    if sample_sparsity < sparsity:
        for polynomial in tqdm(list_of_polynomials):
            sparsity_indices = np.random.choice(nCr(n+d, d), nCr(n+d, d)-sample_sparsity, replace=False)
            for sparsity_index in sparsity_indices:                            
                polynomial[sparsity_index] = 0


list_of_monomial_identifiers_string = [''.join(str(e) for e in monomial_identifier) for monomial_identifier in list_of_monomial_identifiers] if n > 1 else [str(monomial_identifier[0]) for monomial_identifier in list_of_monomial_identifiers]
polynomials_list_df = pd.DataFrame(data=list_of_polynomials, columns=list_of_monomial_identifiers_string).astype('float64')
    
print(len(list_of_monomial_identifiers))
print(polynomials_list_df.shape)

  0%|          | 0/101 [00:00<?, ?it/s]

816
(101, 816)


In [10]:
def gen_regression_symbolic(polynomial, m=None,n_samples=100,n_features=2,noise=0.0,noise_dist='normal', seed=42):
    
    import numpy as np
    from sympy import Symbol,sympify
    from sklearn.datasets import make_friedman1
    from sklearn.preprocessing import MinMaxScaler
    
    np.random.seed(seed)
    random.seed(seed)  
    
    lst_features, evals = make_friedman1(n_samples=n_samples, n_features=n_features, noise=noise, random_state=seed)
    
    evals = MinMaxScaler().fit_transform(evals.reshape(-1, 1))
    
    return polynomial, lst_features, evals

In [11]:
if same_training_all_lambda_nets:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
    result_list = parallel(delayed(gen_regression_symbolic)(list_of_polynomials[i],
                                                            n_samples=lambda_dataset_size,
                                                            n_features=n,
                                                               noise=noise,
                                                               noise_dist=noise_distrib, 
                                                               seed=RANDOM_SEED) for i in range(polynomial_data_size))  
else:
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='multiprocessing')
    result_list = parallel(delayed(gen_regression_symbolic)(list_of_polynomials[i],
                                                            n_samples=lambda_dataset_size,
                                                            n_features=n,
                                                               noise=noise,
                                                               noise_dist=noise_distrib, 
                                                               seed=RANDOM_SEED+i) for i in range(polynomial_data_size))

X_data_list = [[pd.Series(result[0],  index=list_of_monomial_identifiers_string), pd.DataFrame(result[1], columns=list(variables[:n]))] for result in result_list]
y_data_list = [[pd.Series(result[0],  index=list_of_monomial_identifiers_string), pd.DataFrame(result[2].astype(float), columns=['result'])] for result in result_list]



[Parallel(n_jobs=10)]: Using backend MultiprocessingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    0.2s
[Parallel(n_jobs=10)]: Done  48 out of 101 | elapsed:    0.4s remaining:    0.4s
[Parallel(n_jobs=10)]: Done 101 out of 101 | elapsed:    0.5s finished


In [12]:
X_data_list[0][0].head(10)

300000000000000   0.000
210000000000000   0.000
201000000000000   0.000
200100000000000   0.000
200010000000000   0.000
200001000000000   0.000
200000100000000   0.000
200000010000000   0.000
200000001000000   0.000
200000000100000   0.000
dtype: float64

In [13]:
X_data_list[1][1].head(10)

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o
0,0.436,0.026,0.550,0.435,0.420,0.330,0.205,0.619,0.300,0.267,0.621,0.529,0.135,0.514,0.184
1,0.785,0.854,0.494,0.847,0.080,0.505,0.065,0.428,0.097,0.127,0.597,0.226,0.107,0.220,0.350
2,0.468,0.202,0.640,0.483,0.505,0.387,0.794,0.580,0.162,0.701,0.965,0.500,0.890,0.342,0.567
3,0.428,0.437,0.777,0.536,0.954,0.544,0.082,0.366,0.851,0.406,0.027,0.247,0.067,0.994,0.971
4,0.800,0.602,0.765,0.169,0.293,0.524,0.357,0.046,0.983,0.441,0.504,0.324,0.260,0.387,0.832
5,0.737,0.379,0.013,0.797,0.269,0.583,0.026,0.662,0.388,0.497,0.415,0.351,0.551,0.973,0.113
6,0.313,0.042,0.738,0.658,0.215,0.417,0.644,0.661,0.170,0.882,0.778,0.134,0.869,0.749,0.799
7,0.543,0.221,0.918,0.592,0.346,0.264,0.914,0.420,0.540,0.608,0.826,0.624,0.177,0.591,0.489
8,0.548,0.700,0.246,0.187,0.111,0.274,0.010,0.629,0.295,0.187,0.095,0.284,0.215,0.286,0.471
9,0.549,0.845,0.989,0.049,0.232,0.643,0.161,0.870,0.217,0.742,0.653,0.799,0.031,0.230,0.705


In [14]:
X_data_list[0][1].head(10)

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o
0,0.417,0.720,0.000,0.302,0.147,0.092,0.186,0.346,0.397,0.539,0.419,0.685,0.204,0.878,0.027
1,0.670,0.417,0.559,0.140,0.198,0.801,0.968,0.313,0.692,0.876,0.895,0.085,0.039,0.170,0.878
2,0.098,0.421,0.958,0.533,0.692,0.316,0.687,0.835,0.018,0.750,0.989,0.748,0.280,0.789,0.103
3,0.448,0.909,0.294,0.288,0.130,0.019,0.679,0.212,0.266,0.492,0.053,0.574,0.147,0.589,0.700
4,0.102,0.414,0.694,0.414,0.050,0.536,0.664,0.515,0.945,0.587,0.903,0.137,0.139,0.807,0.398
5,0.165,0.928,0.348,0.751,0.726,0.883,0.624,0.751,0.349,0.270,0.896,0.428,0.965,0.663,0.622
6,0.115,0.949,0.450,0.578,0.408,0.237,0.903,0.574,0.003,0.617,0.327,0.527,0.886,0.357,0.909
7,0.623,0.016,0.929,0.691,0.997,0.172,0.137,0.933,0.697,0.066,0.755,0.754,0.923,0.712,0.124
8,0.020,0.026,0.028,0.246,0.860,0.539,0.553,0.842,0.124,0.279,0.586,0.970,0.561,0.019,0.801
9,0.233,0.807,0.388,0.864,0.747,0.556,0.136,0.060,0.121,0.045,0.107,0.226,0.713,0.560,0.013


In [15]:
y_data_list[0][0].head(10)

300000000000000   0.000
210000000000000   0.000
201000000000000   0.000
200100000000000   0.000
200010000000000   0.000
200001000000000   0.000
200000100000000   0.000
200000010000000   0.000
200000001000000   0.000
200000000100000   0.000
dtype: float64

In [16]:
y_data_list[1][1].head(10)

,result
0,0.213
1,0.600
2,0.352
3,0.591
4,0.494
5,0.758
6,0.298
7,0.504
8,0.439
9,0.560


In [17]:
y_data_list[0][1].head(10)

,result
0,0.586
1,0.330
2,0.488
3,0.475
4,0.189
5,0.562
6,0.371
7,0.549
8,0.371
9,0.637


In [19]:
path_polynomials = './data/saved_polynomial_lists/polynomials_sample_' + path_identifier_polynomial_data + '.csv'
polynomials_list_df.to_csv(path_polynomials, index=False)

path_X_data = './data/saved_polynomial_lists/X_sample_' + path_identifier_polynomial_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)#, protocol=2)
    
path_y_data = './data/saved_polynomial_lists/y_sample_' + path_identifier_polynomial_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)#, protocol=2)
